In [ ]:
import wandb
from wandb.keras import WandbCallback
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Masking, Bidirectional
from keras.callbacks import EarlyStopping
from keras import utils
from keras.utils.vis_utils import plot_model
import matplotlib.pyplot as plt

<h1>You can change the DataSets for Training & Testing but keep the same references for each</h1


In [ ]:
trainX = pd.read_csv('./Train/trainX1.csv')
trainY = pd.read_csv('./Train/trainY1.csv')
testX = pd.read_csv('./Test/testX1.csv')
testY = pd.read_csv('./Test/testY1.csv')

In [ ]:
# Create model

def plot_roc_curve(fpr,tpr): 
  plt.plot(fpr,tpr) 
  plt.axis([0,1,0,1]) 
  plt.xlabel('False Positive Rate') 
  plt.ylabel('True Positive Rate') 
  plt.show()    
  

for dropout in [0.1]:
    #wandb.init(reinit=True)
    print("\n\nTesting model with dropout = %f\n"%dropout)
    model = Sequential()
    model.add(Masking(mask_value=-10, input_shape=(541,1)))
    model.add(Bidirectional(LSTM(units=32)))
    model.add(Dropout(dropout))
    model.add(Dense(1, activation='sigmoid'))
    
    checkpoint_filepath = 'weights.{epoch:02d}-{accuracy:.2f}-{val_accuracy:.2f}.h5'
    
    model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='accuracy',
        mode='max',
        save_best_only=True)
    
    #early_stopper = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=False)
        
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy','AUC'])
    history = model.fit(trainX, trainY, epochs=2000, batch_size=81, validation_split=0.1, verbose=1, callbacks= [model_checkpoint_callback])
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val', 'Loss', 'Val_Loss'], loc='upper left')
    plt.show()
    model.save('model.h5')
    #wandb.save('model.h5')
    #wandb.finish()

<h1>For best results, choose the weights returning the best accuracy on training and valiation</h1>

In [ ]:
model.load_weights(PLACE BEST WEIGHTS HERE)
# example : model.load_weights('./weights.377-0.85-0.78.h5')

In [ ]:
import sklearn
from sklearn.metrics import roc_curve,roc_auc_score

prediction_prob = np.round(model.predict(testX),2)
validation = testY.values.tolist()
validation = np.array(validation)
validation = validation.flatten()
auc_score=roc_auc_score(validation,prediction_prob)
print("With a dropout of",dropout,"The AUC score is {:0.2f}".format(auc_score),"on test set")
#wandb.log({"ROC":plot_roc_curve(fpr,tpr)})   
fpr , tpr , thresholds = roc_curve (validation , prediction_prob)
plot_roc_curve(fpr,tpr)

In [ ]:
comparison = {'prob': prediction_prob.flatten(), 'label': validation}
comparison = pd.DataFrame(comparison)
comp = comparison.sort_values(by='prob')
pd.set_option('display.max_columns', None)
comp.T

In [ ]:
from sklearn.metrics import brier_score_loss

loss = brier_score_loss(comp['label'],comp['prob'])
prediction = list(map(lambda x: 0 if x<0.5 else 1, prediction_prob))

print("With a dropout of 0.1, The AUC score is {:0.2f}".format(auc_score),"on test set")
print("The accuracy on test set is {:0.2f}".format(sum([1 for i in range(len(prediction)) if prediction[i] == validation[i]])/len(prediction)*100))
print('The Brier Score on test set is {:0.2f}.' .format(loss))

In [ ]:

# Prototype of implementation of B-LSTM model



def fibonacci_sequence(list):
    # If the result is above 10, log it with the message : 'Earthquake Risk is High, please evacuate immediately!'
    n = 2
    result = []
    fibonacci = [0, 1, 1, 2, 3, 5, 8, 13, 21,34,55,89,144,233,377,610,987,1597,2584,4181]
    for i in list:
        if i == 0:
            if n > 2:
                print(fibonacci[n-1])
                result.append(fibonacci[n-1])
                if fibonacci[n-1] > 10:
                    print("Earthquake Risk is High, please evacuate immediately!")
                n -= 1	
            else:
                n = 2
                print(fibonacci[n])
                result.append(fibonacci[n])
                if fibonacci[n] > 10:
                    print("Earthquake Risk is High, please evacuate immediately!")
        elif i == 1:
            print(fibonacci[n+1])
            result.append(fibonacci[n+1])
            if fibonacci[n+1] > 10:
                print("Earthquake Risk is High, please evacuate immediately!")
            n += 1
        else:
            print("Invalid input")

    plt.plot(result)
    plt.xticks(range(len(list)), list)
    plt.xlabel('Model Output')
    plt.ylabel('Fibonacci Sequence')
    plt.axhline(y=10, color='orange', linestyle='-')
    plt.axvline(x=len(list)-1, color='red', linestyle='-')
    plt.legend(['Estimated Risk', 'Alert Threshold','Earthquake' ])
    return result

list = comp['label']
fibonacci_sequence(list)

